Importujemy najpierw potrzebne pakiety

In [2]:
#http://www.cs.uregina.ca/Links/class-info/325/PythonSound/
#http://www.cs.usfca.edu/~brooks/S08classes/cs107/sound.html
#http://www.openbookproject.net/py4fun/wave/wave.html
#http://www.informit.com/articles/article.aspx?p=2355856&seqNum=5

import numpy as np
import scipy.io.wavfile
import scipy.signal

def wczytywanie_piosenki(nazwa_piosenki = "song.txt"):

    """
    wczytuje zawartosc pliku song.txt do macierzy numpy-owej
    
    arg:
        str: nazwa_piosenki - nazwa pliku tekstowego wraz z rozszerzeniem (zakladam, tak jak w opisie 
                              projektu, ze bedziemy korzystac z domyslnej wartosci, tj. "song.txt", wiec
                              nie sprawdzam poprawnosci typu tego argumentu; problemem z brakiem tego pliku
                              etc. zajmie sie juz python, wyrzucajac komunikat, wiec tez pomijam)
    
    wyjscie:
        numpy.ndarray (str: U2): song - macierz wymiaru 1 na k (ilosc wierszy w pliku song.txt), zawierajaca
                                        kolejne wiersze z wejsciowego pliku zapisane jako stringi
                                    
        
    """
    
    import numpy as np

    song = np.genfromtxt(nazwa_piosenki,dtype='str')
    
    return song

a = wczytywanie_piosenki()

In [3]:
def wczytywanie_sciezek(lista_trackow):
    """
    
    wczytuje na podstawie pliku "song.txt" (lub innego, ktory definiował liste trackow) zawartosc plikow 
    "trackXY.txt" do jednej macierzy numpy-owej 
    
    arg:
        numpy.ndarray (str: U2): lista_trackow - macierz wymiaru 1 na k, zawierajaca zapisane jako napisy,
                                                 numery kolejnych sciezek do odtworzenia
    
    wyjscie:
        tuple: numpy.ndarray (str: U2): piosenka - macierz, zawierajaca jako wiersze kolejne cwiercnuty utworu,
                                            w kazdym wierszu informacja czy dany sample ma zagrac czy nie,
                                            zapisana jako napis (albo numer sampla albo --)
               bool: dobre_trackusie - zmienna kontrolujaca poprawnosc wczytywanych trackow (metrum i ilosc
                                       sampli)
                                            
    """
    
    import numpy as np
    
    # zapisujemy unikalne numery trackow, ktore maja byc wczytane
    jakie_sciezki = np.unique(lista_trackow)

    # slownik, ktorego kolejne elementy to unikalne sciezki (key) wraz z ich zawartoscia (value)
    sciezki = {}
    
    # zmienna do sprawdzenia czy zachowane jest odpowiednie metrum (liczba wierszy podzielna przez 4) w kazdej
    # ze sciezek, jesli nie, to zwracana macierz bedzie zerowa
    czy_4_na_4 = True
    
    # zmienna do sprawdzenia czy w kazdym tracku uzyto tyle samo sampli, jesli nie, zwracana macierz bedzie
    # zerowa
    czy_tyle_samo_sampli = True
    
    # lista zawierajaca ilosc uzytych sampli w kolejnych trackach
    ile_sampli = []
    
    # cale to definiowanie slownika i szukanie unikalnych trackow po to by raz wczytac zawartosc uzywanych
    # trackow a potem sie do niej odwolywac, a nie wczytywac za kazdym razem od nowa zawartosc trackow
    
    for ktora_sciezka in jakie_sciezki: # w iteratorze bierzemy napisy "01" "02" itd. stringi!!!
        
        # tworzymy napis z nazwa pliku tracka, np. "track01.txt"
        plik = ''.join(['track',ktora_sciezka,'.txt'])
        # wczytujemy zawartosc danego tracka do odpowiednio nazwanego elementu w slowniku sciezki
        sciezki[ktora_sciezka] = np.genfromtxt(plik, dtype='str')
        # dodajemy ilosc kanalow danego tracka do listy
        ile_sampli.append(sciezki[ktora_sciezka].shape[1])
        # sprawdzamy podzielnosc wierdzy danego tracka przez 4, jesli nie zachodzi, drukujemy odpowiedni
        # komunikat i zmieniamy wartosc zmiennej logicznej czy_4_na_4 na False
        if(sciezki[ktora_sciezka].shape[0] % 4 != 0):
            print("Plik: ", plik, " nie ma podzielnej przez 4 liczby wierszy")
            czy_4_na_4 = False

    
    # sprawdzamy czy ilosc uzytych sampli jest taka sama w kazdym tracku
    if(len(set(ile_sampli))!=1): # set-em wyciagamy unikalne wartosci i sprawdzamy czy jest tylko jedna taka
        print("Rozna liczba uzytych sampli w trackach")
        czy_tyle_samo_sampli = False
    
    # jedna zmienna odpowiadajaca za poprawnosc trackow
    dobre_trackusie = czy_4_na_4 and czy_tyle_samo_sampli
    
    # jesli metrum i ilosc sampli sie zgadza zapisujemy zawartosc sciezek w piosence do numpy.ndarraya
    if(dobre_trackusie):
 
            # do zapisu poslugujemy sie lista i funkcja append na liscie, gdyz jest to bardziej efektywne
            # niz append dla numpy.ndarraya, gdy nie znamy jego ostatecznych rozmiarow
            piosenka = []
            
            # 
            if(lista_trackow.size ==1):
                piosenka.append(sciezki[str(lista_trackow)]) #dodajemy odpowiednia wartosc ze slownika sciezki

                #tworzymy z listy obiekt typu numpy.ndarray
                piosenka = np.array(piosenka)
                # obiekt ten zawiera kilka podmacierzy, wiec tworzymy z niego 1 macierz
                piosenka = np.vstack(piosenka)
            else:
                # tym razem iterujemy po kolejnych trackach z lista_trackow, nie po unikalnnych 
                # nazwach trackow
                for ktora_sciezka in lista_trackow:
                    piosenka.append(sciezki[ktora_sciezka]) #dodajemy odpowiednia wartosc ze slownika sciezki

                #tworzymy z listy obiekt typu numpy.ndarray
                piosenka = np.array(piosenka)
                # obiekt ten zawiera kilka podmacierzy, wiec tworzymy z niego 1 macierz
                piosenka = np.vstack(piosenka)
    # gdy nie ma odpowiedniego metrum lub ilosci sampli zwracamy zerowa macierz      
    else: 
        piosenka = np.array([0])

    return piosenka, dobre_trackusie 

pios, k = wczytywanie_sciezek(a)

In [4]:
def wczytywanie_ustawien(plik_konfiguracyjny = "defs.txt"):
    """ 
    wczytywanie pliku z ustawieniami (pliku defs.txt) do slownika
    
    arg:
        str: plik_konfiguracyjny - nazwa pliku konfiguracyjnego z podanymi wartosciami parametrow (tempo itd.)
        
    wyjscie:
        dict: parametry - zapisane nazwy i wartosci uzywanych parametrow
    
    """
    import re
    
    # wczytuje zawartosc pliku (bez pierwszej i ostatniej linijki, jeden wiersz wyjsciowej macierzy, zawiera
    # nazwe parametru i jego wartosc, jako oddzielne elementy, zapisane jako stringi)
    ustawienia = np.genfromtxt(plik_konfiguracyjny, dtype = str, \
                               skip_header=1, skip_footer=1, delimiter=":")
    
    # tworze slownik, ktory bedzie przechowywal wartosci
    parametry = {}
    
    # pozbywam się "" z key i przerabiam value na inty
    
    # jesli mamy 1 parametr (1 linijka w pliku, to ustawienia to zmienna o shape = (2,), 
    # wiec odwoluje sie bezposrednio do zmiennej ustawienia
    if ustawienia.shape == (2,): 
        parametry[re.sub('"','',ustawienia[0])] = ustawienia[1]
    else: # jak mamy wiecej parametrow odwoluje sie do kolejnych linijek macierzy ustawienia
        for l in ustawienia:  
            parametry[re.sub('"','',l[0])] = l[1]
    
    # zamieniamy napisy na odpowiednie wartosci - kontroluje jakie sa parametry, wiec robie to recznie
    try:
        parametry['bpm'] = int(parametry['bpm']) # tempo
        parametry['freq'] = int(parametry['freq']) # frekwencja wyjsciowego utworu
        parametry['loud'] = float(parametry['loud'] ) # glosnosc
        parametry['wages'] = [float(s) for s in parametry['wages'].split(",")] # lista wag dla sampli
    
    # jak nie podano danego parametru to olej
    except KeyError:
        pass
    
    return parametry
b = wczytywanie_ustawien("defs.txt")
b

{'bpm': 120, 'freq': 44100, 'loud': -0.2, 'wages': [0.2, 0.4, 0.3, 0.1]}

In [96]:
if "bpm" in b:
    print(b['bpm'])
else:
    print("bu")

120


In [10]:
def tworzenie_piosenki(macierz_piosenki, czy_pelna = True, bpm = 120, \
                       freq = 44100, wages = None, loud = 0):
    """
    glowna funkcja generujaca cala piosenke
    
    arg:
        numpy.ndarray (str: U2): macierz_piosenki - macierz zawierajaca definicje kolejnych cwiercnut 
                                (co ma byc grane w danej cwiercnucie)
        bool: czy_pelna - zmienna sprawdzajaca czy macierz_piosenki jest zapisana (nie jest, gdy tracki mialy
                          nieodpowiednia liczbe wierszy lub kolumn)
        int: bpm - tempo piosenki w jednostce bpm
        int: freq - ilosc probek w jednej sekundzie
        list (float): wages - wagi kolejnych sampli (jakie znaczenie ma miec 1 probka, 2 etc.)
        float: loud - procent glosnosci, 0 - tak jak oryginalne probki, 1 - na maxa, -1 - sciszamy na maxa
    
    wyjscie:
        numpy.ndarray (numpy.int16): gotowy utwór
    """
    
    
    # macierz piosenki byla pusta, piosenka nie zostala utworzona
    if(czy_pelna == False):
        print("Nie utworzono piosenki")
        return None 
    
    else:
    
        import numpy as np
        import scipy.io.wavfile
        
        t_cwiercnuty = 60 / bpm # czas trwania jednej cwiercnuty (zalezy od tempa)
        ile_cwiercnut = macierz_piosenki.shape[0] # ilosc cwiercnut
        kanaly = macierz_piosenki.shape[1] # ilosc uzywanych sampli
        frekw = freq
        czas_utworu = ile_cwiercnut*t_cwiercnuty
        ilosc_probek = int(frekw*czas_utworu) # ile elementow bedzie w nowym utworze
        
        # bedziemy tylko raz wczytywac zawartosc sampleXY.wav, wiec potrzebuje unikalne numery sampli
        rozne_sample = np.unique(macierz_piosenki) # bierze lacznie z "--"
        
        # w slownikach zapiszemy parametry tych sampli
        sample_co = {} # slownik z wartosciami danego sampla (tj. macierze numpy-owe z amplitudami)
        sample_frekw = {} # slownik z ich frekwencjami
        sample_dl = {} # slownik z ich dlugosciami
        
        #wczytujemy te sample
        for ktory_sampel in rozne_sample: # w iteratorze bierzemy napisy "01" "02"  "--" itd. stringi!!!
        
            if(ktory_sampel != '--'):
                # tworzymy napis z nazwa pliku sampla, np. "sample01.wav"
                plik = ''.join(['sample',ktory_sampel,'.wav'])
                # wczytujemy zawartosc i frekwencje danego sampla do odpowiednio nazwanego elementu 
                # w slowniku sample_co i sample_frekw
                sample_frekw[ktory_sampel], sample_co[ktory_sampel] = scipy.io.wavfile.read(plik)
                # tworzymy mono z naszego sampla
                sample_co[ktory_sampel] = np.mean(sample_co[ktory_sampel], axis=1)/32767
                # normalizujemy te wartosci
                sample_co[ktory_sampel] = np.int16(sample_co[ktory_sampel]/max(np.abs(sample_co[ktory_sampel]))\
                                                   *32767)
                # zapisujemy dlugosc sampli, czyli ilosc probek ( = czas_trwania*frekwencja)
                sample_dl[ktory_sampel] = sample_co[ktory_sampel].shape[0]
                
            else: # to samo robimy dla "--" recznie ustawiamy
                sample_co[ktory_sampel] = np.zeros((1,), dtype=np.int16) # robimy cisze, gdy --
                sample_frekw[ktory_sampel] = frekw # taka sama jak domyslna
                sample_dl[ktory_sampel] = 0 # zakladamy czas 0 sekund
         

        
        
 
        if wages is None:
            wages = np.ones((1,kanaly))  
        else:
            wages = np.array(wages).reshape(1,kanaly) # zeby mialo wymiar (1,kanaly), a nie (kanaly,)      
        
        # definicja nowego utworu
        T = np.linspace(0, czas_utworu, ilosc_probek)
        
        for wiersz in range(0, ile_cwiercnut): # ZMIEN TU NA ile_cwiercnut ZAMIAST 10!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

            sample = [] # wczytamy sample z danej cwiecnuty
            dlugosci = [] # tu zapiszemy ich dlugosci w tej cwiercnucie

            for i in range(0, kanaly):
                
                sampus = macierz_piosenki[wiersz,i]
                sample.append(sample_co[sampus])   
                dlugosci.append(sample_dl[sampus])

                    
            # bierzemy najdluzszy sample i w calosci bedziemy go odtwarzac; reszte zatem tez w calosci
            # odtworzymy, a gdy sie skoncza damy cisze (zera)
            maksik = max(dlugosci)
            # mamy tutaj macierz 4 na max dlugosc, przygotowana do zlaczenia potem tych dzwiekow w jeden 
            pusty = np.int16(np.zeros((len(sample), maksik)))

            # dodajemy nasze dzwieki do tej pustej
            for k in range(0, kanaly):
                pusty[k][0:dlugosci[k]] = sample[k]

           
            # mnozymy kolejne elementy wektora pusty (czyli sample) przez wagi i sumujemy
            cwiercnuta = np.dot(wages, pusty) 
            #otrzymamy wymiar (1, x), a chcemy (x,), wiec bierzemy pierwszy element
            cwiercnuta = cwiercnuta[0]
                        
            # jesli dodanie ostatnich cwiercnut bedzie wiazalo sie z przekroczeniem dlugosci tworzonego
            # utworu, obcinamy ostatnie dzwieki, tak by zmiescic sie w tej dlugosci
            
            # poczatek biezacej cwiercnuty 
            poczatek_cwiercnuty = int(wiersz*t_cwiercnuty*frekw)
            
            # jesli dodanie ostatnich cwiercnut bedzie wiazalo sie z przekroczeniem dlugosci tworzonego
            # utworu, obcinamy ostatnie dzwieki, tak by zmiescic sie w tej dlugosci
            if (poczatek_cwiercnuty + maksik) > ilosc_probek:
                
                T[poczatek_cwiercnuty:(poczatek_cwiercnuty + maksik)]=\
                cwiercnuta[0:len(T[poczatek_cwiercnuty:(poczatek_cwiercnuty + maksik)])]
                
            else:
                T[poczatek_cwiercnuty:(poczatek_cwiercnuty + maksik)] += cwiercnuta
        
        T= np.array(T, dtype=np.int16)
        
        #ustalamy glosnosc utworu
        T = zmiana_glosnosci(T, loud)

        return T

pios, k = wczytywanie_sciezek(a)
#wierszyk = tworzenie_piosenki(pios, k, bpm = b['bpm'], freq = b['freq'], wages = b['wages'])
wierszyk = tworzenie_piosenki(pios, k, **b)
wierszyk 

/home/rexamine/anaconda/lib/python3.4/site-packages/scipy/io/wavfile.py:172: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


array([   12,   -32,    25, ..., -3404, -3481, -3498], dtype=int16)

In [11]:
#zglasnianie utworu

def zmiana_glosnosci(utwor, procent = 0):
    """
    zmienia glosnosc utworu (jego amplitudy)
    
    arg:
        numpy.ndarray (numpy.int16): utwor - dzwiek, ktory ma byc zglosniony lub zciszony
        
        float: procent - liczba obrazujaca zmiane glosnosci utworu, osiaga wartosci od -1 do 1
                         dla 0 brak zmian, dla 1 - "100% glosniej", dla -1 "100% ciszej"
    
    wyjscie:
        numpy.ndarray (numpy.int16): glosniejszy - sciszony lub zglosniony utwor
    """
    if(-1 <= procent <= 1):
        #ile razy mamy pomnozyc amplitude naszego dzwieku
        mnoznik = 0
        if( procent < 0 ):
            mnoznik = 1 + procent
        else:
            # obliczamy najwyzsza amplitude w danym utworze i ona bedzie wyznaczac jak bardzo mozemy podglosnic
            maks_ampli = 0
            maks_ampli = max(abs(utwor))
            mnoznik = 32767/maks_ampli # maksymalny mnoznik
            # mnoznik minimalnie moze osiagnac wartosc 1, to co powyzej (mnoznik-1) mnozymy o procent zglosnienia
            # i dodajemy do podstawy (czyli 1)
            mnoznik = 1 + (mnoznik - 1)*procent
        glosniej = mnoznik * utwor
        glosniej = glosniej.astype(np.int16) #glosniej = np.array(glosniej, dtype=np.int16)
        return glosniej
    else:
        print("Podaj procent z zakresu -1 do 1")
                               

wierszyk1 = zmiana_glosnosci(wierszyk, b['loud'])
wierszyk1


array([    9,   -25,    20, ..., -2723, -2784, -2798], dtype=int16)

In [17]:

def zapis_utworu(utwor, nazwa = 'test1.wav', freq = 44100):
    import numpy as np
    import scipy.io.wavfile
    import scipy.signal

    scipy.io.wavfile.write(nazwa, 
                           freq,
                       utwor
                        )
zapis_utworu(wierszyk, freq=b['freq'])

In [18]:
!!aplay /home/rexamine/Desktop/test1.wav

["Playing WAVE '/home/rexamine/Desktop/test1.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Mono"]

In [264]:
def f(k= True,bpm=0,freq=1,loud=2,wages=None):
    print(k,bpm,freq,loud,wages)

def sprawdz(f, **kwargs):
    try:
        f(**kwargs)
        print("Zaden blad nie wyskoczyl, a wynik to:")
        return f(**kwargs)
    # dla ifa z raise ValueError
    except ValueError:
        return "Blad wartosci"
    # do assertow
    except AssertionError:
        return 'Assert wyskoczyl'

In [251]:
b

{'bpm': 120, 'freq': 44100, 'loud': -0.2, 'wages': [0.2, 0.4, 0.3, 0.1]}

In [11]:
kg = {'bpm': 111}
kg

{'bpm': 111}

In [266]:
f(False,**kg) # musze miec tak samo nazwane argumenty w funkcji jak te parametry

False 111 1 2 None


In [5]:
import zipfile

zip = zipfile.ZipFile('utwor1.zip')
zip.extractall()
zip.close()

In [71]:
from encodings.aliases import aliases
#aliases.items()